In [7]:
"""

Basic clustering of CDIAC data files from Globus

"""

import matplotlib.pyplot as plt
from collections import Counter
import binascii
import os
from pathlib import Path

file_path = "pagman.txt"
#test file path and experiment with a .tsv file 
# feel free to change because this is my machine specific

f = open(file_path, "rb")
contents = f.read()
c = Counter(contents)

print("Metadata associated with the file: ")

split_tuple = os.path.splitext(file_path)
print("File extension: ", split_tuple[1])
print("File name: ", split_tuple[0])
print("File path: ", os.path.abspath(file_path))
print("File size is: ", os.path.getsize(file_path), "bytes")
print("Parent Directory: ", os.path.dirname(os.path.abspath(file_path)))


print("Here is a list of the 10 most frequent bytes: ")

data = []
values = []

for value, frequency in c.most_common(10):
        data.append(frequency)
        values.append(str(value))
        print("0x{:02x}: {}".format(value, frequency))

print(values)
plt.bar(range(len(values)), data, align='center', tick_label=values)
plt.show()

Metadata associated with the file: 
File extension:  .txt
File name:  pagman
File path:  /home/matthewchen/MetaDataExtractionModel/pagman.txt
File size is:  8026 bytes
Parent Directory:  /home/matthewchen/MetaDataExtractionModel
Here is a list of the 10 most frequent bytes: 
0xe2: 2608
0xa0: 1645
0x80: 1453
0xa3: 963
0xbf: 801
0x0a: 199
0xb6: 87
0x89: 86
0xa4: 84
0x9b: 71
['226', '160', '128', '163', '191', '10', '182', '137', '164', '155']


TypeError: object of type 'int' has no len()